# <B> Deploy models using the framework (Torch) </B>
* Contatiner: conda_pytorch_p39

# Inference scratch

### 본 워크샵의 모든 노트북은 `conda_pytorch_p39` 여기에서 작업 합니다.

이 노트북은 아래와 같은 작업을 합니다.

- 0. 개념 확인
- 1. 환경 셋업
- 2. 모델 아티펙트 다운로드 및 압축해제
- 3. 추론 함수 로컬 테스트
- 4. 로컬 엔드포인트 생성
- 5. 로컬 추론
- 6. 로컬 엔드 포인트 삭제


### 참고: 
- 세이지 메이커 개발자 가이드 --> [추론을 위한 모델 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/deploy-model.html)
- 세이지 메이커 배포에 대한 웹비나 --> [Amazon SageMaker 기반 사전 훈련된 딥러닝 모델 손쉽게 배포하기 – 김대근:: AWS Innovate 2021](https://www.youtube.com/watch?v=ZdOcrLKow3I)
- 세이지 메이커 호스팅 기본 컨셉 --> [SageMaker 호스팅 아키텍쳐](https://github.com/gonsoomoon-ml/SageMaker-Pipelines-Step-By-Step/blob/main/scratch/8.1.Deploy-Pipeline.ipynb)
- 세이지 메이커로 파이토치 사용 --> [Use PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html)
    
---  

# Concept check
![nn](images/inference_container.png)

**[소스]: 위에 "참조" 에 언급된 "세이지 메이커 배포에 대한 웹비나" 의 내용입니다.**

AWS가 관리하는 배포 컨테이너에 대해 좀 더 자세히 살펴 보겠습니다. 각 프레임워크별에 적합한 배포 컨테이너들이 사전 빌드되어 있으며, 텐서플로는 텐서플로 서빙, 파이토치는 torchserve, MXNet은 MMS, scikit learn은 Flask가 내장되어 있습니다. 이 중에서 파이토치의 경우, 기존에는 MMS가 내장되어 있었지만, 2020년 말부터 Amazon과 facebook이 공동으로 개발한 torchserve를 내장하기 시작했습니다. 

배포 컨테이너를 구동할 때에는 추론을 위한 http 요청을 받아들일 수 있는 RESTful API를 실행하는 serve 명령어가 자동으로 실행되면서 엔드포인트가 시작됩니다. 엔드포인트를 시작할 때, SageMaker는 도커 컨테이너에서 사용 가능한 외부의 모델 아티팩트, 데이터, 그리고 기타 환경 설정 정보 등을 배포 인스턴스의 /opt/ml 폴더로 로딩합니다. 
/opt/ml 폴더는 도커 컨테이너의 밖에서 접근 가능한 머신 러닝 관련 파일들의 볼륨을 의미하며, 이 경로에 대한 내용을 알고 있으면 디버깅에 도움이 됩니다.

물론, 훈련과 호스팅을 모두 지원하는 단일 도커 이미지 빌드도 가능하기 때문에 오픈 소스 도커파일을 참조해서 여러분만의 환경을 구성하는 것도 가능합니다.


![model_artifact_structure.png](images/model_artifact_structure.png)

**[소스]: 위에 "참조" 에 언급된 "세이지 메이커 배포에 대한 웹비나" 의 내용입니다.**

우선 TensorFlow의 경우는 model 경로를 별도로 생성해 줘야 합니다. Model의 하위 폴더에는 모델 버전 번호 폴더를 생성한 다음 그 안에 훈련된 모델 파일을 넣으시면 됩니다. 이 때 주의하실 점이 하나 있는데, pb 파일은 SavedModel만 사용하셔야 하며, Frozen Graph 파일은 SavedModel로 변환해서 사용하셔야 합니다.
PyTorch, MXNet은 디렉토리 및 파일 구조가 유사합니다. 별도의 폴더를 생성하지 않고 곧바로 모델 파라메터 파일을 넣으시면 됩니다. 

그리고 code 폴더는 추론에 필요한 스크립트 파일과 requirements.txt를 포함하면 되는데, PyTorch와 MXNet은 굳이 이 폴더를 포함하지 않아도 됩니다. <br>

- inference.py, requirements.txt는 sagemaker.pytorch.model.PyTorchModel의 클래스 생성자에 entry_point, source_dir을 통해서 제공이 됩니다. 이와 같의 스크립트 모드 방식으로 제공이 가능합니다.<br>


![inference_handler.png](images/inference_handler.png)

**[소스]: 위에 "참조" 에 언급된 "세이지 메이커 배포에 대한 웹비나" 의 내용입니다.**

이제, 추론에 필요한 Python 스크립트 인터페이스를 살펴 보겠습니다. 이 인터페이스들은 SageMaker Inference Toolkit이 정의한 인터페이스로, 텐서플로를 제외한 프레임워크들에서 공용으로 사용됩니다. 

model_fn() 함수는 S3나 model zoo에 저장된 모델을 추론 인스턴스의 메모리로 로드 후, 모델을 리턴하는 방법을 정의하는 전처리 함수입니다.

input_fn() 함수는 사용자로부터 입력받은 내용을 모델 추론에 적합하게 변환하는 전처리 함수로, content_type 인자값을 통해 입력값 포맷을 확인할 수 있습니다.

predict_fn() 함수는 추론 함수로, model_fn()에서 리턴받은 모델과 input_fn()에서 변환된 데이터로 추론을 수행합니다.

output_fn() 함수는 추론 결과를 반환하는 후처리 함수입니다.

그리고 input_fn(), predict_fn(), output_fn()을 각각 구현하는 대신, 세 함수들을 한꺼번에 묶어서 transform() 함수에 구현하는 것도 가능합니다. 
처음에는 익숙치 않을 수 있지만 AWS 공식 예제들이나 레퍼런스의 제 핸즈온을 한 번 해 보시면 금방 익숙해질 수 있습니다.

## AutoReload

In [99]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. parameter store 셋팅

In [100]:
import boto3
from utils.ssm import parameter_store

In [101]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

## 2. deploy

### 2.1 Check functions in inference.py with local env.
[중요] inference.py를 만들어 주어야 함
* model_fn: 학습한 모델 로드
* input_fn: endpoint invocation시 전달 되는 input 처리 하는 함수
* predict_fn: forword propagation, input_fn의 이후 호출 
* output_fn: 유저에게 결과 전달

- 사용자 정의 inference 코드를 정의해서 사용하기 전에, 노트북에서 사전 테스트 및 디버깅을 하고 진행하면 빠르게 추론 개발을 할수 있습니다.
- 디폴트 inference code (input_fn, model_fn, predict_fn, output_fn) 을 사용해도 되지만, 상황에 따라서는 사용자 정의가 필요할 수 있습니다. 디폴트 코드는 아래 링크를 참고 하세요.
    - [Deploy PyTorch Models](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#deploy-pytorch-models)
    - [디폴트 inference Code](https://github.com/aws/sagemaker-pytorch-inference-toolkit/blob/master/src/sagemaker_pytorch_serving_container/default_pytorch_inference_handler.py)

### 로컬 모드 수행시, 새로운 로컬모드 수행을 위해서는 이전 사용했던 도커는 반드시 stop 해줘야 한다
* docker ps -a 로 현재 수행중인 contatiner ID 확인 후
* docker stop "<<contatiner ID>>"
* docker container prune -f

* 2.1.1 학습된 모델 경로 이동 (S3 -> Local)

In [102]:
artifact_path = pm.get_params(key=strPrefix + "S3-MODEL-ARTIFACT")
model_data_dir = "./model/"

In [103]:
%%sh -s {artifact_path} {model_data_dir}

artifact_path=$1
model_data_dir=$2

echo $artifact_path
echo $model_data_dir

# 기존 데이터 삭제
rm -rf $model_data_dir/*

# 모델을 S3에서 로컬로 다운로드
aws s3 cp $artifact_path $model_data_dir

# 모델 다운로드 폴더로 이동
cd $model_data_dir

# 압축 해제
tar -xvf model.tar.gz 

s3://sagemaker-ap-northeast-2-419974056037/byom-model-output/pytorch-training-2023-02-16-01-22-09-857/output/model.tar.gz
./model/
download: s3://sagemaker-ap-northeast-2-419974056037/byom-model-output/pytorch-training-2023-02-16-01-22-09-857/output/model.tar.gz to model/model.tar.gz
model.pth


* 2.1.2. "model_fn"

In [104]:
from source.inference import model_fn

In [105]:
model = model_fn(model_data_dir)

* 2.1.2. "input_fn"

In [106]:
import json
from source.inference import input_fn
from utils.cifar_utils import test_data_loader

In [107]:
test_loader = test_data_loader()
dataiter = iter(test_loader)
images, _ = dataiter.next()
print (f"Image size: {images.size()}")

Image size: torch.Size([1, 3, 32, 32])


In [108]:
data = images.detach().cpu().numpy()
dtype = data.dtype
shape = data.shape
payload={"INPUT": data.tolist(), "SHAPE":shape, "DTYPE": str(dtype)}
payload_json = json.dumps(payload)

In [109]:
input_data = input_fn(request_body=payload_json, request_content_type="application/json")
input_data

string
{"INPUT": [[[[0.23921573162078857, 0.24705886840820312, 0.29411768913269043, 0.301960825920105, 0.2549020051956177, 0.22352945804595947, 0.2705882787704468, 0.24705886840820312, 0.23921573162078857, 0.24705886840820312, 0.2627451419830322, 0.2549020051956177, 0.2627451419830322, 0.301960825920105, 0.32549023628234863, 0.3333333730697632, 0.30980396270751953, 0.2705882787704468, 0.2549020051956177, 0.2549020051956177, 0.22352945804595947, 0.16862750053405762, 0.17647063732147217, 0.16078436374664307, 0.16862750053405762, 0.12156867980957031, 0.09803926944732666, 0.10588240623474121, 0.12156867980957031, 0.07450985908508301, -0.011764705181121826, -0.09019607305526733], [0.19215691089630127, 0.18431377410888672, 0.24705886840820312, 0.301960825920105, 0.2705882787704468, 0.2549020051956177, 0.2862745523452759, 0.2705882787704468, 0.27843141555786133, 0.22352945804595947, 0.21568632125854492, 0.24705886840820312, 0.27843141555786133, 0.3333333730697632, 0.34117650985717773, 0.34117

tensor([[[[ 0.2392,  0.2471,  0.2941,  ...,  0.0745, -0.0118, -0.0902],
          [ 0.1922,  0.1843,  0.2471,  ...,  0.0667, -0.0196, -0.0667],
          [ 0.1843,  0.1843,  0.2392,  ...,  0.0902,  0.0196, -0.0588],
          ...,
          [-0.4667, -0.6706, -0.7569,  ..., -0.7020, -0.8980, -0.6863],
          [-0.5216, -0.6157, -0.7255,  ..., -0.7961, -0.7725, -0.8431],
          [-0.5765, -0.5608, -0.6471,  ..., -0.8118, -0.7333, -0.8353]],

         [[-0.1216, -0.1294, -0.0902,  ..., -0.2549, -0.2863, -0.3333],
          [-0.1216, -0.1373, -0.1059,  ..., -0.2549, -0.2863, -0.3098],
          [-0.1373, -0.1451, -0.1294,  ..., -0.2314, -0.2549, -0.3020],
          ...,
          [-0.0275, -0.2157, -0.3098,  ..., -0.2392, -0.4980, -0.3333],
          [-0.0902, -0.2000, -0.3333,  ..., -0.3569, -0.3569, -0.4980],
          [-0.1608, -0.1765, -0.3020,  ..., -0.3961, -0.3412, -0.4745]],

         [[-0.6157, -0.6314, -0.6000,  ..., -0.7176, -0.7176, -0.7412],
          [-0.6000, -0.6863, -

* 2.1.3 predict_fn

In [110]:
from source.inference import predict_fn

In [111]:
pred = predict_fn(input_data, model)

* 2.1.4 output_fn

In [112]:
from source.inference import output_fn

In [113]:
output_fn(pred)

'{"pred": [[-1.1902360916137695, -1.0329632759094238, -0.1275089979171753, 1.8876615762710571, -1.063999056816101, 0.5308530926704407, 0.1034134179353714, -1.643244743347168, 2.1620473861694336, -0.5270442366600037]]}'

### 2.2 deploy (create an endpoint) in local env

* set instance types  

In [114]:
import os
import subprocess
try:
    if subprocess.call("nvidia-smi") == 0: instance_type = "local_gpu"
    else:instance_type = "local"        
except:pass
print("Instance type = " + instance_type)

Thu Feb 16 01:37:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1B.0 Off |                    0 |
| N/A   29C    P0    25W /  70W |    969MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:00:1C.0 Off |                    0 |
| N/A   

* deploy

In [115]:
import time
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.serializers import CSVSerializer, NumpySerializer, JSONSerializer
from sagemaker.deserializers import JSONDeserializer, NumpyDeserializer

In [116]:
strEndpointName = f"endpoint-{strPrefix}{int(time.time())}"
print (f"Endpoint-name: {strEndpointName}")

Endpoint-name: endpoint-SM-BYOM-1676511437


In [117]:
local_esimator = PyTorchModel(
    source_dir="./source/serve",
    entry_point="inference.py",
    model_data=pm.get_params(key=strPrefix + "S3-MODEL-ARTIFACT"),
    role=pm.get_params(key=strPrefix + "SAGEMAKER-ROLE-ARN"),
    framework_version='1.12.1',
    py_version='py38',
    model_server_workers=1,
)

In [118]:
local_predictor = local_esimator.deploy(
    endpoint_name=strEndpointName,
    instance_type=instance_type, 
    initial_instance_count=1,
    serializer=JSONSerializer('application/json'), ## 미적용 시 default: application/x-npy, boto3 기반 invocation시 무시
    deserializer=JSONDeserializer('application/json'), ## 미적용 시 default: application/x-npy, boto3 기반 invocation시 무시
    wait=True,
    log=True,
)

Attaching to 0r8lbnqeqg-algo-1-dc588
0r8lbnqeqg-algo-1-dc588 | Collecting sagemaker
0r8lbnqeqg-algo-1-dc588 |   Downloading sagemaker-2.132.0.tar.gz (668 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.0/668.0 kB 29.7 MB/s eta 0:00:0031m? eta -:--:--
0r8lbnqeqg-algo-1-dc588 |   Preparing metadata (setup.py) ... done
0r8lbnqeqg-algo-1-dc588 | Requirement already satisfied: boto3 in /opt/conda/lib/python3.8/site-packages (from -r /opt/ml/model/code/requirements.txt (line 2)) (1.26.53)
0r8lbnqeqg-algo-1-dc588 | Requirement already satisfied: botocore in /opt/conda/lib/python3.8/site-packages (from -r /opt/ml/model/code/requirements.txt (line 3)) (1.29.53)
0r8lbnqeqg-algo-1-dc588 | Requirement already satisfied: pandas in /opt/conda/lib/python3.8/site-packages (from -r /opt/ml/model/code/requirements.txt (line 4)) (1.5.3)
0r8lbnqeqg-algo-1-dc588 | Collecting nvgpu
0r8lbnqeqg-algo-1-dc588 |   Downloading nvgpu-0.9.0-py2.py3-none-any.whl (9.4 kB)
0r8lbnqeqg-algo-1-dc588 | Collecting attr

* inference (invocation)

In [119]:
import json
from source.inference import input_fn
from utils.cifar_utils import test_data_loader

In [120]:
test_loader = test_data_loader()
dataiter = iter(test_loader)
images, _ = dataiter.next()
print (f"Image size: {images.size()}")

0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,540 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,540 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]166
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,541 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,541 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,544 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,549 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9000.
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:30,551 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1676511450551
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37

In [121]:
data = images.detach().cpu().numpy()
dtype = data.dtype
shape = data.shape
payload={"INPUT": data.tolist(), "SHAPE":shape, "DTYPE": str(dtype)}
#payload_json = json.dumps(payload)

In [122]:
pred_results = local_predictor.predict(payload)
pred_results

0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,677 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1676511460677
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,680 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Backend received inference at: 1676511460
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,680 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - ### input_fn ###
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,681 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - content_type: application/json
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,681 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - string
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,681 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - ### predict_fn ###
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:40,682 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - #### type of input data: <class 'torch.Tensor'>
0r8lbnqeqg-algo-1-dc588 | 2023-02-16T01:37:42,246 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - ### output_fn ###
0r8lb

{'pred': [[-1.1902360916137695,
   -1.0329632759094238,
   -0.1275089979171753,
   1.8876615762710571,
   -1.063999056816101,
   0.5308530926704407,
   0.1034134179353714,
   -1.643244743347168,
   2.1620473861694336,
   -0.5270442366600037]]}

### 2.3 deploy (create an endpoint) in cloud

In [123]:
instance_type = "ml.g4dn.xlarge"

In [124]:
strEndpointName = f"endpoint-{strPrefix}{int(time.time())}"
print (f"Endpoint-name: {strEndpointName}")

Endpoint-name: endpoint-SM-BYOM-1676511467


In [125]:
cloud_esimator = PyTorchModel(
    source_dir="./source/serve",
    entry_point="inference.py",
    model_data=pm.get_params(key=strPrefix + "S3-MODEL-ARTIFACT"),
    role=pm.get_params(key=strPrefix + "SAGEMAKER-ROLE-ARN"),
    framework_version='1.12.1',
    py_version='py38',
    model_server_workers=1,
)

In [126]:
cloud_predictor = cloud_esimator.deploy(
    endpoint_name=strEndpointName,
    instance_type=instance_type, 
    initial_instance_count=1,
    serializer=JSONSerializer('application/json'),
    deserializer=JSONDeserializer('application/json'),
    wait=True,
    log=True,
)

--

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sagemaker/local/image.py", line 854, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sagemaker/local/image.py", line 916, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/sagemaker/local/image.py", line 859, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp9zay3m45/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited 

0r8lbnqeqg-algo-1-dc588 exited with code 137
Aborting on container exit...
----------!

In [134]:
pm.put_params(key=strPrefix + "ENDPOINT-NAME", value=strEndpointName, overwrite=True)

'Store suceess'

* 2.3.1 Inference by SageMaker SDK

In [135]:
test_loader = test_data_loader()
dataiter = iter(test_loader)
images, _ = dataiter.next()
print (f"Image size: {images.size()}")

Image size: torch.Size([1, 3, 32, 32])


In [136]:
data = images.detach().cpu().numpy()
dtype = data.dtype
shape = data.shape
payload={"INPUT": data.tolist(), "SHAPE":shape, "DTYPE": str(dtype)}

In [137]:
%%time
pred_results = cloud_predictor.predict(payload)
pred_results

CPU times: user 19.8 ms, sys: 2.6 ms, total: 22.4 ms
Wall time: 77.8 ms


{'pred': [[-1.1902360916137695,
   -1.0329632759094238,
   -0.1275089979171753,
   1.8876615762710571,
   -1.063999056816101,
   0.5308530926704407,
   0.1034134179353714,
   -1.643244743347168,
   2.1620473861694336,
   -0.5270442366600037]]}

* 2.3.2 inference by boto3
    - serialization을 customization 할 수 있음 (아래 pickle based serialization 참조)

In [138]:
import boto3
import sagemaker

### boto3 기반 invocation시 runtime_client가 필요함
* 이때 엔드포인트의 위치 (local/cloud)를 반드시 구분해 줘야함

In [139]:
if instance_type == 'local_gpu': runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
else: runtime_client = boto3.Session().client('sagemaker-runtime')
runtime_client

In [140]:
def invoke_endpoint(runtime_client, endpoint_name, payload, content_type):
    response = runtime_client.invoke_endpoint(
        EndpointName=endpoint_name, 
        ContentType=content_type, 
        Body=payload,
    )

    result = response['Body'].read().decode().splitlines()    
    
    return result

* json based serialization

In [141]:
import json

### [중요] boto3로 inference 할 때는 payload를 반드시 serialization 해서 넣어줘야 함
* sagemaker SDK는 deploy시 셋팅한 방식대로 serialization을 자동으로 해 준다
* boto3는 사용자가 직접 serialization을 한 후 inference 한다. (대신 serialization 자유도가 있다)

In [142]:
payload_dump = json.dumps(payload)

In [143]:
start_time = time.time()
result = invoke_endpoint(
    runtime_client=runtime_client,
    endpoint_name=pm.get_params(key=strPrefix + "ENDPOINT-NAME"), 
    payload=payload_dump,
    content_type='application/json'
)

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

--- 0.1368875503540039 seconds ---
result:  ['{"pred": [[-1.1902360916137695, -1.0329632759094238, -0.1275089979171753, 1.8876615762710571, -1.063999056816101, 0.5308530926704407, 0.1034134179353714, -1.643244743347168, 2.1620473861694336, -0.5270442366600037]]}']


* pickle based serialization

In [144]:
import pickle

In [145]:
test_loader = test_data_loader()
dataiter = iter(test_loader)
images, _ = dataiter.next()
print (f"Image size: {images.size()}")

Image size: torch.Size([1, 3, 32, 32])


### pickle 기반 serialization시 numpy, tensor등을 별도 변환없이 그대로 전송할 수 있음
* 아래 INPUT의 경우 json으로 serialization 할 때 처럼 .tolist()를 할 필요 없음

In [146]:
data = images.detach().cpu().numpy()
dtype = data.dtype
shape = data.shape
payload={"INPUT": data, "SHAPE":shape, "DTYPE": str(dtype)} 

In [147]:
payload_dump = pickle.dumps(payload)

In [148]:
%%time

result = invoke_endpoint(
    runtime_client=runtime_client,
    endpoint_name=pm.get_params(key=strPrefix + "ENDPOINT-NAME"), 
    payload=payload_dump,
    content_type='application/pickle'
)

print('result: ', result)

result:  ['{"pred": [[-1.1902360916137695, -1.0329632759094238, -0.1275089979171753, 1.8876615762710571, -1.063999056816101, 0.5308530926704407, 0.1034134179353714, -1.643244743347168, 2.1620473861694336, -0.5270442366600037]]}']
CPU times: user 2.96 ms, sys: 3.77 ms, total: 6.73 ms
Wall time: 45.7 ms
